# Topic 2. Parameter estimates. Criteria for consent

# Source code for all datasets

In [18]:
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from scipy.stats import norm, expon, poisson, chisquare, sem, t, chi2
from typing import *

In [19]:
with open('../config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

In [48]:
def chi_square_test(data, dist_type="normal", bins=10, **kwargs):
    """
    Perform Chi-Square test for a given dataset and distribution type.

    Parameters:
    data : array-like
        Input data to test.
    dist_type : str
        Type of distribution ('normal', 'exponential', 'poisson').
    bins : int
        Number of bins to divide data.
    **kwargs :
        Parameters for the chosen distribution:
            - For 'normal': mean, std (default: calculated from data).
            - For 'exponential': scale (1/lambda, default: calculated from data).
            - For 'poisson': mu (default: calculated from data).

    Returns:
    chi2_stat : float
        Chi-Square statistic.
    p_value : float
        p-value of the test.
    """
    # Calculate histogram (observed frequencies)
    observed_freq, bin_edges = np.histogram(data, bins=bins, density=False)
    n = len(data)

    # Choose distribution and calculate expected frequencies
    if dist_type == "normal":
        mu = kwargs.get("mean", np.mean(data))
        std = kwargs.get("std", np.std(data))
        expected_freq = [
            n * (norm.cdf(bin_edges[i + 1], loc=mu, scale=std) - norm.cdf(bin_edges[i], loc=mu, scale=std))
            for i in range(len(bin_edges) - 1)
        ]

    elif dist_type == "exponential":
        scale = kwargs.get("scale", 1 / np.mean(data))
        expected_freq = [
            n * (expon.cdf(bin_edges[i + 1], scale=scale) - expon.cdf(bin_edges[i], scale=scale))
            for i in range(len(bin_edges) - 1)
        ]

    elif dist_type == "poisson":
        mu = kwargs.get("mu", np.mean(data))
        expected_freq = [
            n * (poisson.cdf(bin_edges[i + 1] - 1, mu) - poisson.cdf(bin_edges[i] - 1, mu))
            for i in range(len(bin_edges) - 1)
        ]

    else:
        raise ValueError("Unsupported distribution type. Choose 'normal', 'exponential', or 'poisson'.")

    expected_freq = np.asarray(expected_freq, dtype=np.float64)
    expected_freq = np.maximum(expected_freq, 1e-10)
    observed_total = np.sum(observed_freq)
    expected_total = np.sum(expected_freq)
    expected_freq *= (observed_total / expected_total)

    # Perform Chi-Square Test
    chi2_stat, p_value = chisquare(f_obs=observed_freq, f_exp=expected_freq)

    return chi2_stat, p_value

In [34]:
def chisq_test_printer(df: pd.DataFrame, param: str, bins: int = 10, dist_type: str = "normal", **kwargs) -> None:
    """
    Perform Chi-Square test for a given dataset and distribution type.
    And print results!
    
    Parameters:
    data : array-like
        Input data to test.
    dist_type : str
        Type of distribution ('normal', 'exponential', 'poisson').
    bins : int
        Number of bins to divide data.
    **kwargs :
        Parameters for the chosen distribution:
            - For 'normal': mean, std (default: calculated from data).
            - For 'exponential': scale (1/lambda, default: calculated from data).
            - For 'poisson': mu (default: calculated from data).

    Returns:
    None
    """
    chi2_stat, p_val = chi_square_test(df[param], bins=bins, dist_type=dist_type, kwargs=kwargs)
    print(f"Chi-Square Statistic for {param} distribution: {chi2_stat}, p-value: {p_val}")
    if p_val > 0.05:
        print(f"The data is likely {dist_type} (fail to reject H0).")
    else:
        print(f"The data is likely not {dist_type} (reject H0).")


In [22]:
def confidence_intervals_normal(data: pd.DataFrame, confidence: float = 0.95) -> Tuple[float, float]:
    """
    Compute confidence intervals for the mean and standard deviation of a normal distribution.

    Parameters:
    data : array-like
        Input data.
    confidence : float
        Confidence level (default is 0.95).

    Returns:
    ci_mean : tuple
        Confidence interval for the mean.
    ci_std : tuple
        Confidence interval for the standard deviation.
    """
    n = len(data)
    mean = np.mean(data)
    std = np.std(data, ddof=1)

    # Confidence interval for the mean
    se = sem(data)
    t_value = t.ppf((1 + confidence) / 2, df=n - 1)
    margin_error_mean = t_value * se
    ci_mean = (mean - margin_error_mean, mean + margin_error_mean)

    # Confidence interval for the standard deviation
    chi2_lower = chi2.ppf((1 - confidence) / 2, df=n - 1)
    chi2_upper = chi2.ppf(1 - (1 - confidence) / 2, df=n - 1)
    ci_std = (
        std * np.sqrt((n - 1) / chi2_upper),
        std * np.sqrt((n - 1) / chi2_lower)
    )

    return ci_mean, ci_std

In [23]:
def conf_int_printer(data: pd.DataFrame, confidence: float = 0.95, text: str = "") -> None:
    """
    Compute confidence intervals for the mean and standard deviation of a normal distribution.
    And print results!

    Parameters:
    data : array-like
        Input data.
    confidence : float
        Confidence level (default is 0.95).
    text: str
        Parameter to print in first print().

    Returns:
    None
    """
    ci_mean, ci_std = confidence_intervals_normal(data, confidence)
    print(f"Parameter: {text if text else 'Unknown'}")
    print(f"Confidence Interval for Mean: {ci_mean}")
    print(f"Confidence Interval for Standard Deviation: {ci_std}")

---
# Dataset: "Babyboom"
**Variables:**
- Time of birth recorded on the 24-hour clock
- Sex of the child (1 = girl, 2 = boy)
- Birth weight in grams
- Number of minutes after midnight of each birth

## 1. Check Normality of Birth Weight
- First, check the normality of birth weights using all data, without separating by sex.
- Then, perform the same check for boys and girls separately.
- Use point estimates of parameters for hypothesis testing.
- Construct confidence intervals for the parameters of the normal distribution.

## 2. Test Hypothesis for Exponential Distribution of Time Between Births
- Test whether the time between births follows an exponential distribution.
- Use point estimates of parameters for hypothesis testing.

## 3. Test Hypothesis for Poisson Distribution of Births per Hour
- Test if the number of births per hour follows a Poisson distribution.
- Use point estimates of parameters for hypothesis testing.

## 0. Preparing data

In [24]:
babyboom_column_specifications = [
    (0, 8),  # Time of birth
    (8, 16),  # Sex of the child
    (16, 24),  # Birth weight in grams
    (24, 32)  # Minutes after midnight
]

babyboom_column_names = [
    "Time_of_birth",
    "Sex_of_child",
    "Birth_weight_grams",
    "Minutes_after_midnight"
]


In [46]:
ds_babyboom = pd.read_fwf(cfg['datasets']['babyboom'], colspecs=babyboom_column_specifications,
                          names=babyboom_column_names)
ds_babyboom.head()

,Time_of_birth,Sex_of_child,Birth_weight_grams,Minutes_after_midnight
0,5,1,3837,5
1,104,1,3334,64
2,118,2,3554,78
3,155,2,3838,115
4,257,2,3625,177


## 1. Check Normality of Birth Weight

For all genders

In [49]:
chisq_test_printer(ds_babyboom, "Birth_weight_grams", bins =8)

Chi-Square Statistic for Birth_weight_grams distribution: 17.776662953218455, p-value: 0.013019333129642109
The data is likely not normal (reject H0).


For boys

In [50]:
chisq_test_printer(ds_babyboom[ds_babyboom['Sex_of_child'] == 2], "Birth_weight_grams", bins =8)

Chi-Square Statistic for Birth_weight_grams distribution: 6.2708745544125755, p-value: 0.5085007396249484
The data is likely normal (fail to reject H0).


For girls

In [51]:
chisq_test_printer(ds_babyboom[ds_babyboom['Sex_of_child'] == 1], "Birth_weight_grams", bins =8)

Chi-Square Statistic for Birth_weight_grams distribution: 10.231846150114865, p-value: 0.1758112504054016
The data is likely normal (fail to reject H0).


### Confidence Intervals for Normal Distribution Parameters

In [38]:
conf_int_printer(ds_babyboom['Birth_weight_grams'], 0.95, "Birth_weight_grams")

Parameter: Birth_weight_grams
Confidence Interval for Mean: (3115.418005028038, 3436.491085881053)
Confidence Interval for Standard Deviation: (436.272478864305, 669.0306102925873)


## 2. Test Hypothesis for Exponential Distribution of Time Between Births

In [52]:
ds_babyboom['time_diff'] = ds_babyboom['Minutes_after_midnight'].diff()
ds_babyboom = ds_babyboom.dropna()
chisq_test_printer(ds_babyboom, "time_diff", dist_type="exponential")

Chi-Square Statistic for time_diff distribution: 58.476190476190474, p-value: 2.6332709959162704e-09
The data is likely not exponential (reject H0).


## 3. Test Hypothesis for Poisson Distribution of Births per Hour

In [55]:

ds_babyboom['hour'] = ds_babyboom['Minutes_after_midnight'] // 60  # Integer division to get the hour (in minutes)

# Count the number of births per hour and merge it back to the original DataFrame
births_per_hour = ds_babyboom.groupby('hour').size().reset_index(name='births_count')
ds_babyboom = pd.merge(ds_babyboom, births_per_hour, on='hour', how='left')

chisq_test_printer(ds_babyboom, "births_count", dist_type="poisson")

Chi-Square Statistic for births_count distribution: 322899815629.22186, p-value: 0.0
The data is likely not poisson (reject H0).



---

# Dataset: "Euroweight"
**Variables:**
- weight
- batch

### 1. Test Hypothesis for Normal Distribution of Coin Weights
- First, combine all coins into a single sample and test for normality of the weight distribution.
- Then, test for normality of the weight distribution within each batch.
- Use point estimates of parameters for hypothesis testing.
- Construct confidence intervals for the parameters of the normal distribution.


---

# Dataset: "iris.txt"
**Variables:**
- Sepal length
- Sepal width
- Petal length
- Petal width
- Class (species of iris)

### 1. Test Hypothesis for Normal Distribution of Flower Lengths by Iris Type
- Test the hypothesis that flower length (sepal and petal lengths) follows a normal distribution for each iris species.
- Use point estimates of parameters for hypothesis testing.
- Construct confidence intervals for the parameters of the normal distribution.